**Please run the code blocks in order or click Run All.**

Importing all the necessary libraries at start of the notebook to give a clean look.

In [1]:
from models.customer_model import CustomerModel
from models.transaction_model import TransactionModel
from utils.arithmetic_helper import ArithmeticHelper
from utils.constants import Constants
from utils.csv_reader import CSVFileUtils
import tabulate as tb
from typing import Dict

## **0. Generate Dataset?**
I have used a synthetic dataset which is generated from another three datasets. I have already generated the dataset and saved it in the 'data' folder. If you want to generate the dataset again, you can run the code block below. It will generate a new dataset in 'data' folder as 'transactions_generated.csv'. Please note that during each generation, the samples will be different as it is picked randomly from the original datasets. Please refresh the IDE's file browser to see the new generated file.

In [2]:
# from utils.synthetic_data_utils import SyntheticDataUtils
# SyntheticDataUtils.generate_synthetic_data()

## **1. Hello Data!**
To load any csv file, I have written a reusable class named CSVFileUtils which has methods to load and save csv files. While loading the csv file, you can specify the file path and the range of rows to load.

In [3]:
raw_csv_data = CSVFileUtils.read_csv_with_rage(file_path=Constants.synth_generated_csv_file_path(),start=1,end=3)
print(tb.tabulate(raw_csv_data,headers="keys",tablefmt="outline"))


+------------+--------------+-----------------------+---------------+----------------+-------------------+----------------+--------------+------------+---------+--------------------+-----------------+-----------------+
|   order_id | order_date   |   days_since_purchase |   customer_id |   customer_age | customer_gender   | product_name   |   product_id |   quantity |   price |   discounted_price | coupon_code     | shipping_city   |
+============+==============+=======================+===============+================+===================+================+==============+============+=========+====================+=================+=================+
|  686800706 | 10/18/2014   |                     0 |          1647 |              0 | None              | T-shirt        |          131 |       8446 |      50 |                  0 | Hooray!! 342AHR | Nebraska        |
|  185941302 | 11/7/2011    |                     0 |          1536 |              0 | None              | Dress          | 


## **2. Pick the Right Container**
dict data-type is basically a key-value pair like a JSON object where a unique key is used to access its values. A named type, we can say it is a tuple with fixed field names and a class is a custom collection of data-types and methods.


## 3. Transaction Class and OO data structure
I have created a `TransactionModel` class in `models/transaction_model.py`. This class contains a `from_raw_data` method which will generate a `TransactionModel` object from the raw data.


## 4.	Bulk Loader

TransactionModel class also contain a `load_transactions` method which will load the transactions from the raw data in the csv file. It will return a list of `TransactionModel` objects.

In [4]:
transactions_list:list[TransactionModel]  = TransactionModel.load_transactions()


## 5. Quick Profiling
I have created a 'ArithmeticHelper' class in 'utils/arithmetic_helper.py' which contains methods to do basic mean, min, max, count operations.

In [5]:
prices = [t.price for t in transactions_list]
cities = [t.shipping_city for t in transactions_list]
print("Min (price): ", ArithmeticHelper.calculate_min(prices))
print("Mean (price): ", ArithmeticHelper.calculate_mean(prices))
print("Max (price): ", ArithmeticHelper.calculate_max(prices))
print("Unique count (city): ", ArithmeticHelper.unique_items_count(cities))


Min (price):  10.0
Mean (price):  56.885
Max (price):  100.0
Unique count (city):  50



## 6. Spot the Grime
1. The dates in the dataset are in different formats.
2. The Price field have values which are not numbers or non parsable numbers.
3. Product name string is not standardized. Can be trimmed and converted to title case, can remove double spaces and special characters.
4. Shipping city names are not standardized. Can be trimmed and converted to title case, can remove double spaces and special characters.



## 7. Cleaning Rules
The `TransactionModel` class has a `clean` method which will clean the transaction data. It will remove the leading and trailing spaces, convert the strings to title case, remove double spaces and special characters from the product name and shipping city name. It will also convert the date to a standard format and parse the price to a float value.

In [6]:
cleaned_transactions_list :list[TransactionModel]= []

for t in transactions_list:
    cleaned_object = t.clean()
    if cleaned_object is not None:
        cleaned_transactions_list.append(cleaned_object)

print("Raw transaction list length", len(transactions_list))
print("Clean transaction list length", len(cleaned_transactions_list))
if len(cleaned_transactions_list) == len(transactions_list):
    print("Hooray!! All transactions are cleaned successfully.")
else:
    print("Some transactions are not cleaned successfully.")


Raw transaction list length 1000
Clean transaction list length 1000
Hooray!! All transactions are cleaned successfully.



## 8. Transformations

The `TransactionModel` class has a `calculate_discount` method which will calculate the discount if there is coupon code present in the transaction object.

In [7]:
for t in cleaned_transactions_list:
    t.calculate_discount()


## 9. Feature Engineering

The `TransactionModel` class has a `calculate_days_since_purchase` method which will calculate the days since the purchase was made.


In [8]:
for t in cleaned_transactions_list:
    t.calculate_days_since_purchase()


## 	10. Mini-Aggregation
The `ArithmeticHelper` class has `calculate_revenue_per_shipping_city` method to calculate revenue by shipping city which also takes discounts into considerations.


In [9]:
city_and_revenue:Dict[str, float] = ArithmeticHelper.calculate_revenue_per_shipping_city(cleaned_transactions_list)
for i in range(3):
    print(f"City: {list(city_and_revenue.keys())[i]}, Revenue: {list(city_and_revenue.values())[i]}")


City: Nebraska, Revenue: 6765010.600000001
City: West Virginia, Revenue: 5846367.399999999
City: New Jersey, Revenue: 5265584.800000001



## 11. Serialization Checkpoint

The `TransactionModel` class has a `to_dict` method which will convert the transaction object to a JSON string. The class also have a `save_transactions_to_json_file` method which will take a list of transaction objects and save them to a JSON file. The `TransactionModel` class also has a `save_transactions_to_parquet` method which will save the transactions to a parquet file.
The files will be saved in the 'data/cleaned' folder'.


In [10]:
TransactionModel.save_transactions_to_json_file(cleaned_transactions_list)
TransactionModel.save_transactions_to_parquet(cleaned_transactions_list)


## 12. Soft Interview Reflection
The OOPs have helped me to create a clean and reusable code structure. I have used the TransactionModel class to encapsulate the transaction data and its methods. The CSVFileUtils class has helped me to load and save the csv files in a reusable way. The ArithmeticHelper class has helped me to do basic arithmetic operations on the transaction data. Overall, I feel that my code is readable, maintainable and reusable and makes more sense to anyone who is reading it.


## 13. Data-Dictionary Section
I am merging customers gender and age from customer dataset to the transactions dataset.

In [11]:
customer_raw_data = CSVFileUtils.read_csv_with_rage(file_path=Constants.customer_csv_file_path(),start=1)
customer_list: list[CustomerModel] = []
for row in customer_raw_data:
    customer_list.append(CustomerModel.from_raw_data(row))

for t in cleaned_transactions_list:
    for c in customer_list:
        if t.customer_id == c.customer_id:
            t.add_customer_info(c)

new_table = tb.tabulate([txn.to_dict() for txn in cleaned_transactions_list[:3]],headers="keys",tablefmt="outline")
print(new_table)

+------------+--------------+-----------------------+---------------+----------------+-------------------+----------------+--------------+------------+---------+--------------------+-----------------+-----------------+
|   order_id | order_date   |   days_since_purchase |   customer_id |   customer_age | customer_gender   | product_name   |   product_id |   quantity |   price |   discounted_price | coupon_code     | shipping_city   |
+============+==============+=======================+===============+================+===================+================+==============+============+=========+====================+=================+=================+
|  686800706 | 2014-10-18   |                  3875 |          1647 |             58 | Male              | Tshirt         |          131 |       8446 |      50 |               45   | Hooray!! 342AHR | Nebraska        |
|  185941302 | 2011-11-07   |                  4951 |          1536 |             36 | Male              | Dress          | 

## Data Dictionary for the final dataset is as follows:

| Field               | Type | Description                                | Source                                   |
|---------------------|------|--------------------------------------------|------------------------------------------|
| order_id            | int  | Unique Id for an order                     | transactions_generated.csv               |
| order_date          | date | Date in which order was placed             | transactions_generated.csv               |
| days_since_purchase | int  | Days till today from order_date            | transactions_generated.csv - calculated  |
| customer_id         | int  | Unique Id for a customer                   | customer.csv                             |
| customer_age        | int  | Age of customer                            | customer.csv                             |
| customer_gender     | enum | Gender of customer                         | customer.csv                             |
| product_name        | str  | Name of product                            | fashion_product.csv                      |
| product_id          | int  | Unique Id for a product                    | fashion_product.csv                      |
| quantity            | int  | Quantity of product in order               | transactions_generated.csv               |
| price               | int  | Price of product                           | transactions_generated.csv               |
| discounted_price    | int  | Price after reducing coupon offer          | transactions_generated.csv  - calculated |
| coupon_code         | str  | A discount code                            | transactions_generated.csv               |
| shipping_city       | str  | Address to which the order is being placed | customer.csv                             |


## References:
- [E-commerce dataset](https://excelbianalytics.com/wp/wp-content/uploads/2017/07/1000-Sales-Records.zip)
- [Customer datasets](https://www.kaggle.com/datasets/bhadramohit/customer-shopping-latest-trends-dataset)
- [Products dataset](https://www.kaggle.com/datasets/bhanupratapbiswas/fashion-products)
- [Python OOPs Concepts](https://www.geeksforgeeks.org/python-oops-concepts/)
- [Python CSV File Handling](https://docs.python.org/3/library/csv.html#module-csv)
- [Parquet](https://pypi.org/project/parquet/)